In [1]:
from pathlib import Path

from collections import defaultdict, Counter

import numpy as np

from itertools import combinations

from functools import reduce

In [2]:
data_path = Path.home() / 'workstation' / 'dev' / 'Advent-of-Code-2020' / 'data' / 'day20_input.txt'

In [3]:
data_path.exists()

True

In [4]:
with open(data_path, 'r') as reader:
    image_input = reader.read().strip()

In [5]:
imgs_w_title = image_input.split('\n\n')

In [6]:
def get_pixels(text): 
    "Translate the '.' and '#' to 0's and 1's"
    bits = {'#': 1, '.': 0}
    return tuple(bits[p] for p in text)

def rotate_ctrclockwise(array):
    return np.rot90(array)

def flip_horizontal(array):
    return np.fliplr(array)

def flip_vertical(array):
    return np.flipud(array)

def get_tile_borders(tile_array):
    return [
        tile_array[:, 0],
        tile_array[:, -1],
        tile_array[0, :],
        tile_array[-1, :]
    ]

In [7]:
image_dict = {}

for image in imgs_w_title:
    title, *image = image.split('\n')
    title = title[:-1] # Getting rid of colon
    tile_id = [int(s) for s in title.split() if s.isdigit()]
    image_dict[tile_id[0]] = np.asarray(list(get_pixels(row) for row in image))

In [8]:
# All possible tile borders. 8 possibilities: original borders + flips

tile_borders = defaultdict(list)

for tile_id, tile in image_dict.items():
    tile_borders[tile_id] = (
        get_tile_borders(tile) +
        list(map(flip_vertical, get_tile_borders(tile)))
    )

In [9]:
common_borders = defaultdict(list)

for id_1, id_2 in combinations(tile_borders.keys(), 2):
    cmn_borders = []
    for bdr in tile_borders[id_1]:
        if any(np.array_equal(bdr, x) for x in tile_borders[id_2]):
            cmn_borders.append(bdr)
    common_borders[(id_1, id_2)] = cmn_borders

#### Part 1

In [10]:
def get_corner_tile_ids(border_pair_dict):
    def get_neighbouring_pairs():
        tiles = []

        for ids in border_pair_dict.keys():
            if border_pair_dict[ids]:
                id_1, id_2 = ids
                tiles.extend([*ids])
    
        return tiles
    
    tiles = get_neighbouring_pairs()
    
    for k, v in Counter(tiles).items():
        if v == 2:
            yield k

In [11]:
reduce(lambda x, y: x*y, list(get_corner_tile_ids(common_borders)))

15670959891893